### How many games does a team needs to win to be crowned champions?

First we will need to import the standing tables

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from library import (
    get_squad_as_index,
    remove_unnamed_cols,
    clean_opp_df,
    clean_main_df,
    merge_dfs,
    create_multiple_season_dfs,
    get_single_season_league_data,
    edit_gk_tables,
    get_seasons_list_of_tables,
    edit_standard_stats_table,
)

In [7]:
df_csv = pd.read_csv("./frames/England/1/2017-2018/standard_data.csv")
df_xls = pd.read_excel("./frames/England/1/2017-2018/standard_data.xlsx")

In [8]:
df_csv.head()

,Unnamed: 0,Performance,Performance.1,Performance.2,Performance.3,Performance.4,Performance.5,Performance.6,Per_90_Minutes,Per_90_Minutes.1,...,Per_90_Minutes.14,Per_90_Minutes.15,Per_90_Minutes.16,Per_90_Minutes.17,Per_90_Minutes.18,Per_90_Minutes.19,Expected.4,Expected.5,Expected.6,Expected.7
0,NaN,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls,Ast,...,G+A-PK_Opp_Opp,xG_Opp_Opp,xAG_Opp_Opp,xG+xAG_Opp_Opp,npxG_Opp_Opp,npxG+xAG_Opp_Opp,xG_Opp_Opp,npxG_Opp_Opp,xAG_Opp_Opp,npxG+xAG_Opp_Opp
1,Arsenal,73,61,69,4,5,NaN,NaN,1.92,1.61,...,2.21,1.26,0.87,2.12,1.13,2.0,47.8,43.1,32.9,75.9
2,Bournemouth,45,36,42,3,3,NaN,NaN,1.18,0.95,...,2.68,1.56,1.15,2.71,1.5,2.65,59.2,56.8,43.7,100.5
3,Brighton,33,25,28,5,7,NaN,NaN,0.87,0.66,...,2.21,1.34,0.9,2.23,1.27,2.17,50.8,48.4,34.1,82.5
4,Burnley,35,26,35,0,0,NaN,NaN,0.92,0.68,...,1.68,1.35,1.0,2.35,1.26,2.26,51.2,47.7,38.0,85.8


In [16]:
df_xls.head()

,Unnamed: 0,Performance,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Per_90_Minutes,Unnamed: 9,...,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Expected.1,Unnamed: 40,Unnamed: 41,Unnamed: 42
0,NaN,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls,Ast,...,G+A-PK_Opp_Opp,xG_Opp_Opp,xAG_Opp_Opp,xG+xAG_Opp_Opp,npxG_Opp_Opp,npxG+xAG_Opp_Opp,xG_Opp_Opp,npxG_Opp_Opp,xAG_Opp_Opp,npxG+xAG_Opp_Opp
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arsenal,73,61,69,4,5,NaN,NaN,1.92,1.61,...,2.21,1.26,0.87,2.12,1.13,2,47.8,43.1,32.9,75.9
3,Bournemouth,45,36,42,3,3,NaN,NaN,1.18,0.95,...,2.68,1.56,1.15,2.71,1.5,2.65,59.2,56.8,43.7,100.5
4,Brighton,33,25,28,5,7,NaN,NaN,0.87,0.66,...,2.21,1.34,0.9,2.23,1.27,2.17,50.8,48.4,34.1,82.5


In [10]:
my_dict = get_single_season_league_data("England", 1, 2020)

In [11]:
df = my_dict.get("standard_data")

In [44]:
df

Performance                              Per_90_Minutes        \
                        Gls Ast G-PK  PK PKatt CrdY CrdR            Gls   Ast   
Arsenal                  53  38   47   6     6   49    5           1.39  1.00   
Aston Villa              52  38   47   5     6   71    4           1.37  1.00   
Brighton                 39  24   33   6     9   49    6           1.03  0.63   
Burnley                  32  20   29   3     3   48    0           0.84  0.53   
Chelsea                  56  38   48   8    10   51    3           1.47  1.00   
Crystal Palace           39  29   36   3     4   56    2           1.03  0.76   
Everton                  45  32   41   4     5   59    2           1.18  0.84   
Fulham                   26  18   23   3     6   67    3           0.68  0.47   
Leeds United             60  45   56   4     4   61    1           1.58  1.18   
Leicester City           64  45   54  10    12   61    0           1.68  1.18   
Liverpool                65  43   59   6     6   40    0           1.71  1.13   
Manchester City          82  55   77   5     9   46    2           2.16  1.45   
Manchester Utd           70  51   60  10    11   64    1           1.84  1.34   
Newcastle Utd            44  26   38   6     7   65    3           1.16  0.68   
Sheffield Utd            19  13   16   3     4   73    3           0.50  0.34   
Southampton              47  33   42   5     6   52    3           1.24  0.87   
Tottenham                66  50   61   5     5   57    2           1.74  1.32   
West Brom                33  20   29   4     4   51    4           0.87  0.53   
West Ham                 60  46   58   2     4   50    3           1.58  1.21   
Wolves                   34  21   30   4     4   55    1           0.89  0.55   

                       ...                                        \
                  G+A  ... G+A-PK_Opp_Opp xG_Opp_Opp xAG_Opp_Opp   
Arsenal          2.39  ...           1.53       1.13        0.84   
Aston Villa      2.37  ...           1.84       1.34        0.82   
Brighton         1.66  ...           1.79       0.93        0.61   
Burnley          1.37  ...           2.58       1.44        1.12   
Chelsea          2.47  ...           1.53       0.80        0.53   
Crystal Palace   1.79  ...           2.89       1.53        1.12   
Everton          2.03  ...           2.03       1.32        0.89   
Fulham           1.16  ...           2.08       1.38        0.98   
Leeds United     2.76  ...           2.11       1.52        1.03   
Leicester City   2.87  ...           1.92       1.21        0.87   
Liverpool        2.84  ...           1.76       1.13        0.71   
Manchester City  3.61  ...           1.05       0.80        0.40   
Manchester Utd   3.18  ...           1.74       1.09        0.75   
Newcastle Utd    1.84  ...           2.53       1.54        1.07   
Sheffield Utd    0.84  ...           2.68       1.64        1.18   
Southampton      2.11  ...           2.71       1.40        0.91   
Tottenham        3.05  ...           1.47       1.29        0.84   
West Brom        1.39  ...           3.11       1.79        1.16   
West Ham         2.79  ...           1.95       1.28        0.96   
Wolves           1.45  ...            NaN        NaN         NaN   

                                                               Expected  \
                xG+xAG_Opp_Opp npxG_Opp_Opp npxG+xAG_Opp_Opp xG_Opp_Opp   
Arsenal                   1.97         1.07             1.91       43.0   
Aston Villa               2.16         1.22             2.04       51.1   
Brighton                  1.54         0.79             1.40       35.3   
Burnley                   2.56         1.38             2.50       54.7   
Chelsea                   1.33         0.71             1.25       30.3   
Crystal Palace            2.66         1.45             2.57       58.2   
Everton                   2.21         1.24             2.12       50.1   
Fulham                    2.37         1.24             2.22 

In [15]:
type(df.columns)

pandas.core.indexes.multi.MultiIndex

In [18]:
df.to_excel("/home/grgkaralis/Documents/file.xlsx", header=df.columns)

In [33]:
df.to_csv("/home/grgkaralis/Documents/file.csv", header=df.columns)

/home/grgkaralis/Documents/f_venv/lib/python3.11/site-packages/pandas/core/internals/blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [60]:
df1 = pd.read_excel("/home/grgkaralis/Documents/file.xlsx", header=[0,1])

In [61]:
df1.head()

Unnamed: 0_level_0 Performance                                    \
  Unnamed: 0_level_1         Gls   Ast  G-PK   PK PKatt  CrdY CrdR   
0                NaN         NaN   NaN   NaN  NaN   NaN   NaN  NaN   
1            Arsenal        53.0  38.0  47.0  6.0   6.0  49.0  5.0   
2        Aston Villa        52.0  38.0  47.0  5.0   6.0  71.0  4.0   
3           Brighton        39.0  24.0  33.0  6.0   9.0  49.0  6.0   
4            Burnley        32.0  20.0  29.0  3.0   3.0  48.0  0.0   

  Per_90_Minutes        ...                                        \
             Gls   Ast  ... G+A-PK_Opp_Opp xG_Opp_Opp xAG_Opp_Opp   
0            NaN   NaN  ...            NaN        NaN         NaN   
1           1.39  1.00  ...           1.53       1.13        0.84   
2           1.37  1.00  ...           1.84       1.34        0.82   
3           1.03  0.63  ...           1.79       0.93        0.61   
4           0.84  0.53  ...           2.58       1.44        1.12   

                                                 Expected               \
  xG+xAG_Opp_Opp npxG_Opp_Opp npxG+xAG_Opp_Opp xG_Opp_Opp npxG_Opp_Opp   
0            NaN          NaN              NaN        NaN          NaN   
1           1.97         1.07             1.91       43.0         40.6   
2           2.16         1.22             2.04       51.1         46.4   
3           1.54         0.79             1.40       35.3         29.8   
4           2.56         1.38             2.50       54.7         52.3   

                                
  xAG_Opp_Opp npxG+xAG_Opp_Opp  
0         NaN              NaN  
1        31.8             72.4  
2        31.1             77.5  
3        23.3             53.2  
4        42.5             94.8  

[5 rows x 43 columns]

In [47]:
df1.columns

MultiIndex([('Unnamed: 0_level_0', 'Unnamed: 0_level_1'),
            (       'Performance',                'Gls'),
            (       'Performance',                'Ast'),
            (       'Performance',               'G-PK'),
            (       'Performance',                 'PK'),
            (       'Performance',              'PKatt'),
            (       'Performance',               'CrdY'),
            (       'Performance',               'CrdR'),
            (    'Per_90_Minutes',                'Gls'),
            (    'Per_90_Minutes',                'Ast'),
            (    'Per_90_Minutes',                'G+A'),
            (    'Per_90_Minutes',               'G-PK'),
            (    'Per_90_Minutes',             'G+A-PK'),
            (    'Per_90_Minutes',                 'xG'),
            (    'Per_90_Minutes',                'xAG'),
            (    'Per_90_Minutes',             'xG+xAG'),
            (    'Per_90_Minutes',               'npxG'),
            ( 

In [48]:
df.columns

MultiIndex([(   'Performance',              'Gls'),
            (   'Performance',              'Ast'),
            (   'Performance',             'G-PK'),
            (   'Performance',               'PK'),
            (   'Performance',            'PKatt'),
            (   'Performance',             'CrdY'),
            (   'Performance',             'CrdR'),
            ('Per_90_Minutes',              'Gls'),
            ('Per_90_Minutes',              'Ast'),
            ('Per_90_Minutes',              'G+A'),
            ('Per_90_Minutes',             'G-PK'),
            ('Per_90_Minutes',           'G+A-PK'),
            ('Per_90_Minutes',               'xG'),
            ('Per_90_Minutes',              'xAG'),
            ('Per_90_Minutes',           'xG+xAG'),
            ('Per_90_Minutes',             'npxG'),
            ('Per_90_Minutes',         'npxG+xAG'),
            (      'Expected',               'xG'),
            (      'Expected',             'npxG'),
            